In [28]:
import requests
import urllib.request

base_url = 'http://karten.guedels.ch/'
img_dir = 'map_images/'
all_maps_query = 'users.php?lastMaps=all'

r = requests.get(base_url)

with open('tmp.html', 'w', encoding='utf-8') as test_file:
    test_file.write(r.text)

In [5]:
html = r.text

In [6]:
import logging
logging.basicConfig(filename='scrape_maps.log', encoding='utf-8', level=logging.ERROR)

In [7]:
with open('test.txt', 'w', encoding='utf-8') as fp:
    fp.write(r.text)

In [8]:
import re
find_digits = re.compile('\d+')

img_numbers = [re.findall(find_digits, match)[0] for match in re.findall('map=\d+', html)]

In [9]:
import os
save_dir = 'maps'
if not os.path.exists('maps'):
    os.mkdir(save_dir)

In [10]:
def download_image(img_url, filename):
    return urllib.request.urlretrieve(img_url, filename)

In [11]:
def try_download_image(base_url, img_dir, save_dir, img, exts):
    for i, ext in enumerate(exts):
        img_url = f'{base_url}{img_dir}{img}{ext}'
        filename = f'{save_dir}/{img}{ext}'
        try:
            if i > 0:
                logging.debug(f'\tRetrying with {ext}... ')
            download_image(img_url, filename)
            return True
        except HTTPError as e:
            logging.debug(f'\t{str(e)}: {filename}')
    return False

In [12]:
def find_imgs_to_downloads(save_dir, imgs, exts):
    imgs_to_exclude = []
    imgs_to_download = []
    for img in imgs:
        file_exists = False
        for ext in exts:
            filename = f'{save_dir}/{img}{ext}'
            file_exists = file_exists or os.path.exists(filename)
        if file_exists:
            imgs_to_exclude.append(img)
        else:
            imgs_to_download.append(img)
    return imgs_to_download, imgs_to_exclude

In [25]:
from urllib.error import HTTPError
import itertools

def download_images(base_url, img_dir, save_dir, imgs):
    exts = ['.jpg', '.png', '.JPG', '.PNG']

    imgs_to_download, imgs_to_exclude  = find_imgs_to_downloads(save_dir, imgs, exts)

    if imgs_to_exclude:
        print(f'Ignoring images: {",".join(imgs_to_exclude)}')

    if imgs_to_download:
        print(f'Downloading images: {",".join(imgs_to_download)}')

    failed_imgs = []
    for img in imgs_to_download:
        success = try_download_image(base_url, img_dir, save_dir, img, exts)
        if not success:
            failed_imgs.append(img)
    if failed_imgs:
        print(f'Failed to download: {",".join(failed_imgs)}')

In [26]:
from time import time

time_start = time()
download_images(base_url, img_dir, save_dir, img_numbers)
time_end = time()
print(f'Took {time_end - time_start:2.4f} seconds to download.')

Ignoring images: 2814,2780,564,2561,2386,2822,285,246,447,2722,251,1057,1935,2713,2818,2816,2711,2814,2822,2821,2820,2819,2818,2817,2816,2815,2814,2813,1304,2324,2328,2325,2266,2165,2087,375,2004,1912
Took 0.0051 seconds to download.
